**Connexion to my Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/testhoggo/

/content/drive/MyDrive/testhoggo


**Transform PDF to Image : Requirement and function**

In [3]:
#Requirement for Imagemagick
!apt-get update
!apt-get install libmagickwand-dev
!sed -i 's/^.*policy.*coder.*none.*PDF.*//' /etc/ImageMagick-6/policy.xml
!pip install wand
!pip install ghostscript

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [423 kB]
Hit:12 http://ppa.launchpad.net

In [4]:
#Function to convert pdf to image using Imagemagick
from wand.image import Image as wi

def pdf_to_image(pdf_path):
  pdf_file = wi(filename=pdf_path, resolution=300)
  Images = pdf_file.convert('PNG')
  ImageSequence = 1

  for img in pdf_file.sequence:
    if (len(pdf_file.sequence)==ImageSequence):
      Image = wi(image = img)
      Image.compression_quality = 90
      Image.alpha_channel = "remove"
      Image.save(filename = "Image.png")
      break
    ImageSequence += 1 

**Image Processing obtained from Imagemagick**


In [5]:
#using openCV 
import cv2
import numpy as np

# get grayscale image
def get_grayscale(image):
  image = cv2.resize(image, None, fx=3, fy=3)
  return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#thresholding
def thresholding(image):
  return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

**Tesseract OCR : Requirement and functions**

In [6]:
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev
!pip install pytesseract
!apt-get install tesseract-ocr-fra  #for french

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 97 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,480 kB/s)
Selecting previously unselect

In [7]:
#useful libraries
from google.colab.patches import cv2_imshow
import pytesseract # OCR library

In [8]:
#remove empty lines in a text
def remove_empty_lines(entree):
  tab = entree.strip()
  tableausansvide = [ x for x in tab.splitlines() if x!='' ]
  res = ''
  for i in range(0, len(tableausansvide)):
    res = res + tableausansvide[i] + '\n'
  return res
#transform text to list
def text_in_list(result):
  return (remove_empty_lines(result).strip().split("\n"))

In [9]:
#get numero de sucurite, nom prenom and organisme de rattachement
def image_ocr(pdf_path):

  config = "--psm 3" #Fully automatic page segmentation , but no OSD. (Default)
  try:
    pdf_to_image(pdf_path)
  except:
    return 0
  image = cv2.imread("Image.png")
  #get name
  x = 70
  y = 250
  w = 1200
  h = 600
  region_Nom = image[y:h, x:w]
  result = pytesseract.image_to_string(thresholding(get_grayscale(region_Nom)), config=config, lang="fra")
  try:
    name = text_in_list(result)[-1]
  except:
    name = ""
  #get num_org
  x = 120
  y = 970
  w = 1000
  h = 1200
  region_Nom = image[y:h, x:w]
  result = pytesseract.image_to_string(thresholding(get_grayscale(region_Nom)), config=config, lang="fra")
  try:
    num_org = [x for x in text_in_list(result) if len(x)==11][0].strip()
  except:
    num_org = ""
  #get num_secu
  x = 1260
  y = 990
  w = 1950
  h = 1300
  region_Nom = image[y:h, x:w]
  result = pytesseract.image_to_string(thresholding(get_grayscale(region_Nom)), config=config, lang="fra")
  try:
    num_sec = [x for x in text_in_list(result) if len(x)==21][0].strip()
  except:
    num_sec=""

  return(num_sec, name, num_org)

In [10]:
#main part
import urllib.request # for pdf file downloading

#just modify url for a new pdf file
url="AttestationDroits_1.pdf" #pdf path from your computer or internet
try:
  #download file from internet
  urllib.request.urlretrieve(url, "filename.pdf")
  pdf_path = "filename.pdf"
except:
  #download file from your computer
  pdf_path = url

data={}
try:
  data["Numero de securite sociale"] ,data["Nom prenoms"], data["Organisme de Rattachement"] = image_ocr(pdf_path)
except:
  print("Error")

#create json file
import json
with open("output.json", "w") as write_file:
  json.dump(data, write_file)